# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2520152850>
├── 'a' --> tensor([[ 0.0999, -0.3764,  0.5572],
│                   [-0.6558,  0.6625, -0.0450]])
└── 'x' --> <FastTreeValue 0x7f2520152820>
    └── 'c' --> tensor([[ 1.5851,  1.8039,  1.8201,  0.5122],
                        [-0.1517, -0.7849,  0.5316,  0.7595],
                        [-0.4146,  0.0526,  0.4069,  0.4538]])

In [4]:
t.a

tensor([[ 0.0999, -0.3764,  0.5572],
        [-0.6558,  0.6625, -0.0450]])

In [5]:
%timeit t.a

76.3 ns ± 0.664 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2520152850>
├── 'a' --> tensor([[-1.0360, -0.1541, -0.7171],
│                   [ 0.1899,  1.0630, -0.2858]])
└── 'x' --> <FastTreeValue 0x7f2520152820>
    └── 'c' --> tensor([[ 1.5851,  1.8039,  1.8201,  0.5122],
                        [-0.1517, -0.7849,  0.5316,  0.7595],
                        [-0.4146,  0.0526,  0.4069,  0.4538]])

In [7]:
%timeit t.a = new_value

78.6 ns ± 0.981 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0999, -0.3764,  0.5572],
               [-0.6558,  0.6625, -0.0450]]),
    x: Batch(
           c: tensor([[ 1.5851,  1.8039,  1.8201,  0.5122],
                      [-0.1517, -0.7849,  0.5316,  0.7595],
                      [-0.4146,  0.0526,  0.4069,  0.4538]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0999, -0.3764,  0.5572],
        [-0.6558,  0.6625, -0.0450]])

In [11]:
%timeit b.a

69.2 ns ± 1.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3867,  0.1422,  0.6732],
               [-0.4165, -0.0045,  0.0119]]),
    x: Batch(
           c: tensor([[ 1.5851,  1.8039,  1.8201,  0.5122],
                      [-0.1517, -0.7849,  0.5316,  0.7595],
                      [-0.4146,  0.0526,  0.4069,  0.4538]]),
       ),
)

In [13]:
%timeit b.a = new_value

654 ns ± 8.41 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.09 µs ± 12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

15.5 µs ± 354 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

298 µs ± 6.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

276 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f246ba53850>
├── 'a' --> tensor([[[ 0.0999, -0.3764,  0.5572],
│                    [-0.6558,  0.6625, -0.0450]],
│           
│                   [[ 0.0999, -0.3764,  0.5572],
│                    [-0.6558,  0.6625, -0.0450]],
│           
│                   [[ 0.0999, -0.3764,  0.5572],
│                    [-0.6558,  0.6625, -0.0450]],
│           
│                   [[ 0.0999, -0.3764,  0.5572],
│                    [-0.6558,  0.6625, -0.0450]],
│           
│                   [[ 0.0999, -0.3764,  0.5572],
│                    [-0.6558,  0.6625, -0.0450]],
│           
│                   [[ 0.0999, -0.3764,  0.5572],
│                    [-0.6558,  0.6625, -0.0450]],
│           
│                   [[ 0.0999, -0.3764,  0.5572],
│                    [-0.6558,  0.6625, -0.0450]],
│           
│                   [[ 0.0999, -0.3764,  0.5572],
│                    [-0.6558,  0.6625, -0.0450]]])
└── 'x' --> <FastTreeValue 0x7f246b408eb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

51.2 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f247cf37f70>
├── 'a' --> tensor([[ 0.0999, -0.3764,  0.5572],
│                   [-0.6558,  0.6625, -0.0450],
│                   [ 0.0999, -0.3764,  0.5572],
│                   [-0.6558,  0.6625, -0.0450],
│                   [ 0.0999, -0.3764,  0.5572],
│                   [-0.6558,  0.6625, -0.0450],
│                   [ 0.0999, -0.3764,  0.5572],
│                   [-0.6558,  0.6625, -0.0450],
│                   [ 0.0999, -0.3764,  0.5572],
│                   [-0.6558,  0.6625, -0.0450],
│                   [ 0.0999, -0.3764,  0.5572],
│                   [-0.6558,  0.6625, -0.0450],
│                   [ 0.0999, -0.3764,  0.5572],
│                   [-0.6558,  0.6625, -0.0450],
│                   [ 0.0999, -0.3764,  0.5572],
│                   [-0.6558,  0.6625, -0.0450]])
└── 'x' --> <FastTreeValue 0x7f25201cc1c0>
    └── 'c' --> tensor([[ 1.5851,  1.8039,  1.8201,  0.5122],
                        [-0.1517, -0.7849,  0.5316,  0.7595],
                 

In [23]:
%timeit t_cat(trees)

47.7 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

86.1 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.5851,  1.8039,  1.8201,  0.5122],
                       [-0.1517, -0.7849,  0.5316,  0.7595],
                       [-0.4146,  0.0526,  0.4069,  0.4538]],
              
                      [[ 1.5851,  1.8039,  1.8201,  0.5122],
                       [-0.1517, -0.7849,  0.5316,  0.7595],
                       [-0.4146,  0.0526,  0.4069,  0.4538]],
              
                      [[ 1.5851,  1.8039,  1.8201,  0.5122],
                       [-0.1517, -0.7849,  0.5316,  0.7595],
                       [-0.4146,  0.0526,  0.4069,  0.4538]],
              
                      [[ 1.5851,  1.8039,  1.8201,  0.5122],
                       [-0.1517, -0.7849,  0.5316,  0.7595],
                       [-0.4146,  0.0526,  0.4069,  0.4538]],
              
                      [[ 1.5851,  1.8039,  1.8201,  0.5122],
                       [-0.1517, -0.7849,  0.5316,  0.7595],
                       [-0.4146,  0.0526,  0.4069,  0.4538]],

In [26]:
%timeit Batch.stack(batches)

111 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.5851,  1.8039,  1.8201,  0.5122],
                      [-0.1517, -0.7849,  0.5316,  0.7595],
                      [-0.4146,  0.0526,  0.4069,  0.4538],
                      [ 1.5851,  1.8039,  1.8201,  0.5122],
                      [-0.1517, -0.7849,  0.5316,  0.7595],
                      [-0.4146,  0.0526,  0.4069,  0.4538],
                      [ 1.5851,  1.8039,  1.8201,  0.5122],
                      [-0.1517, -0.7849,  0.5316,  0.7595],
                      [-0.4146,  0.0526,  0.4069,  0.4538],
                      [ 1.5851,  1.8039,  1.8201,  0.5122],
                      [-0.1517, -0.7849,  0.5316,  0.7595],
                      [-0.4146,  0.0526,  0.4069,  0.4538],
                      [ 1.5851,  1.8039,  1.8201,  0.5122],
                      [-0.1517, -0.7849,  0.5316,  0.7595],
                      [-0.4146,  0.0526,  0.4069,  0.4538],
                      [ 1.5851,  1.8039,  1.8201,  0.5122],
                   

In [28]:
%timeit Batch.cat(batches)

203 µs ± 2.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

532 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
